In [20]:
import xgboost as xgb
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import csv

# 2. 加载

In [30]:
# 加载并完善特征数据
x, y = load_data('/Users/cap/data/titanic/train.csv', True)
print(x.shape, y.shape)

随机森林预测缺失年龄：--start--
随机森林预测缺失年龄：--over--
(4455, 9) (4455,)


# 3. 划分训练集和测试集

In [37]:
# x样本特征集，y样本结果, test_size样本占比, random_state随机数种子
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=1)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(2227, 9) (2228, 9) (2227,) (2228,)


# 4. 预测

In [21]:
def show_accuracy(a, b, tip):
    acc = a.ravel() == b.ravel()
    acc_rate = 100 * float(acc.sum()) / a.size
    return acc_rate

In [22]:
def load_data(file_name, is_train):
    data = pd.read_csv(file_name)  # 数据文件路径
    # print 'data.describe() = \n', data.describe()
 
    # 性别 将性别字段Sex中的值 female用0，male用1代替,类型 int
    data['Sex'] = data['Sex'].map({'female': 0, 'male': 1}).astype(int)
 
    # 补齐船票价格缺失值
    if len(data.Fare[data.Fare.isnull()]) > 0:
        fare = np.zeros(3)
        for f in range(0, 3):
            fare[f] = data[data.Pclass == f + 1]['Fare'].dropna().median()
        for f in range(0, 3):  # loop 0 to 2
            data.loc[(data.Fare.isnull()) & (data.Pclass == f + 1), 'Fare'] = fare[f]
 
    # 年龄：使用均值代替缺失值
    # mean_age = data['Age'].dropna().mean()
    # data.loc[(data.Age.isnull()), 'Age'] = mean_age
    if is_train:
        # 年龄：使用随机森林预测年龄缺失值
        print('随机森林预测缺失年龄：--start--') 
        data_for_age = data[['Age', 'Survived', 'Fare', 'Parch', 'SibSp', 'Pclass']]
        age_exist = data_for_age.loc[(data.Age.notnull())]   # 年龄不缺失的数据
        age_null = data_for_age.loc[(data.Age.isnull())]
        # print 'data_for_age=\n', data_for_age
        # print 'age_exis=\n', age_exist
        # print 'age_null=\n',age_null
        # print age_exist
        x = age_exist.values[:, 1:]
        y = age_exist.values[:, 0]
        # print 'x = age_exist.values[:, 1:] 中 x=',x
        # print 'y = age_exist.values[:, 0] 中 y=',y
        #n_estimators 决策树的个数，越多越好,值越大，性能就会越差,但至少100
        rfr = RandomForestRegressor(n_estimators=1000)
        rfr.fit(x, y)
        age_hat = rfr.predict(age_null.values[:, 1:])
        # print age_hat
        # print 'age_hat',age_hat
        #填充年龄字段中值为空的
        data.loc[(data.Age.isnull()), 'Age'] = age_hat
        print('随机森林预测缺失年龄：--over--')
    else:
        print('随机森林预测缺失年龄2：--start--') 
        data_for_age = data[['Age', 'Fare', 'Parch', 'SibSp', 'Pclass']]
        age_exist = data_for_age.loc[(data.Age.notnull())]  # 年龄不缺失的数据
        age_null = data_for_age.loc[(data.Age.isnull())]
        # print age_exist
        x = age_exist.values[:, 1:]
        y = age_exist.values[:, 0]
        rfr = RandomForestRegressor(n_estimators=1000)
        rfr.fit(x, y)
        age_hat = rfr.predict(age_null.values[:, 1:])
        # print age_hat
        data.loc[(data.Age.isnull()), 'Age'] = age_hat
        print('随机森林预测缺失年龄2：--over--') 
 
    # 起始城市
    data.loc[(data.Embarked.isnull()), 'Embarked'] = 'S'  # 保留缺失出发城市
    # print data['Embarked']
    embarked_data = pd.get_dummies(data.Embarked)
    # print embarked_data
    embarked_data = embarked_data.rename(columns=lambda x: 'Embarked_' + str(x))
    data = pd.concat([data, embarked_data], axis=1)
    # print data.describe()
    data.to_csv('New_Data.csv')
 
    x = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]
    # x = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
    y = None
    if 'Survived' in data:
        y = data['Survived']
 
    x = np.array(x)
    y = np.array(y)
 
    x = np.tile(x, (5, 1))
    y = np.tile(y, (5, ))
    if is_train:
        return x, y
    return x, data['PassengerId']

In [23]:
def write_result(c, c_type):
    file_name = '/Users/cap/data/titanic/titanic.test.csv'
    x, passenger_id = load_data(file_name, False)
 
    if type == 3:
        x = xgb.DMatrix(x)
    y = c.predict(x)
    y[y > 0.5] = 1
    y[~(y > 0.5)] = 0
 
    predictions_file = open("Prediction_%d.csv" % c_type, "wb")
    open_file_object = csv.writer(predictions_file)
    open_file_object.writerow(["PassengerId", "Survived"])
    open_file_object.writerows(zip(passenger_id, y))
    predictions_file.close()

In [24]:
def totalSurvival(y_hat,tip):
    total=0
    for index,value in enumerate(y_hat):
        if value==1:
            total=total+1
    print(tip,'存活：',total) 
    print('人') 

## logistic回归

In [38]:
lr = LogisticRegression(penalty='l2')
lr.fit(x_train, y_train)
y_hat = lr.predict(x_test)
lr_rate = show_accuracy(y_hat, y_test, 'Logistic回归 ')
totalSurvival(y_hat,'Logistic回归')

Logistic回归 存活： 810
人


/Users/cap/opt/anaconda3/envs/cap/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## 随机森林 

In [39]:
# n_estimators：决策树的个数,越多越好,不过值越大，性能就会越差,至少100
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(x_train, y_train)
y_hat = rfc.predict(x_test)
rfc_rate = show_accuracy(y_hat, y_test, '随机森林 ')
totalSurvival(y_hat,'随机森林')
# write_result(rfc, 2)

随机森林 存活： 860
人


## XGBoost

In [13]:
data_train = xgb.DMatrix(x_train, label=y_train)
data_test = xgb.DMatrix(x_test, label=y_test)
watch_list = [(data_test, 'eval'), (data_train, 'train')]
param = {'max_depth': 6, 'eta': 0.8, 'silent': 1, 'objective': 'binary:logistic'}
bst = xgb.train(param, data_train, num_boost_round=100, evals=watch_list)
y_hat = bst.predict(data_test)
y_hat[y_hat > 0.5] = 1
y_hat[~(y_hat > 0.5)] = 0
xgb_rate = show_accuracy(y_hat, y_test, 'XGBoost ')
totalSurvival(y_hat,'xgboost')

[0]	eval-error:0.15754	train-error:0.129771
[1]	eval-error:0.149461	train-error:0.110912
[2]	eval-error:0.135548	train-error:0.09789
[3]	eval-error:0.124776	train-error:0.090705
[4]	eval-error:0.102334	train-error:0.072744
[5]	eval-error:0.090215	train-error:0.060171
[6]	eval-error:0.071813	train-error:0.040413
[7]	eval-error:0.067774	train-error:0.037719
[8]	eval-error:0.065978	train-error:0.035025
[9]	eval-error:0.069569	train-error:0.035923
[10]	eval-error:0.063285	train-error:0.033229
[11]	eval-error:0.058348	train-error:0.029187
[12]	eval-error:0.058348	train-error:0.029187
[13]	eval-error:0.05386	train-error:0.024697
[14]	eval-error:0.05386	train-error:0.024697
[15]	eval-error:0.052513	train-error:0.023799
[16]	eval-error:0.050718	train-error:0.02335
[17]	eval-error:0.048923	train-error:0.022901
[18]	eval-error:0.047576	train-error:0.022003
[19]	eval-error:0.044434	train-error:0.020656
[20]	eval-error:0.04623	train-error:0.021105
[21]	eval-error:0.047127	train-error:0.020207
[22]

In [14]:
print('Logistic回归：%.3f%%' % lr_rate)
print('随机森林：%.3f%%' % rfc_rate) 
print('XGBoost：%.3f%%' % xgb_rate) 

Logistic回归：78.815%
随机森林：97.935%
XGBoost：97.756%
